In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_folder = './data'

test = pd.read_csv(f'{data_folder}/test.csv')
test['seq_len'] = test.protein_sequence.str.len()
test.head()

,seq_id,protein_sequence,pH,data_source,seq_len
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,221
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,221
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,220
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,221
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,221


In [3]:
test_wt = 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTN' \
    'AILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'

# Process results

## MutCompute

In [19]:
import math


class ProtStability:
    """
        Predict stability of Protein mutants based on MutCompute.
    """
    def __init__(self, prob_path='data/mc/DNase_novozyme.csv', wildtype=None):
        self.aa3to1 = {'ala': 'A', 'arg': 'R', 'asn': 'N', 'asp': 'D', 'asx': 'B', 'cys': 'C', 'glu': 'E', 'gln': 'Q',
                       'glx': 'Z', 'gly': 'G', 'his': 'H', 'ile': 'I', 'leu': 'L', 'lys': 'K', 'met': 'M', 'phe': 'F',
                       'pro': 'P', 'ser': 'S', 'thr': 'T', 'trp': 'W', 'tyr': 'Y', 'val': 'V'}
        df_stability = pd.read_csv(prob_path)
        # Replace 0 with 1e-5, only for juexiao's output
        for aa in ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']:
             df_stability['pr'+aa] = df_stability['pr'+aa].replace(0, 1e-5)

        self.pos2aa2prob = {df_stability.iloc[i].pos: {
            self.aa3to1[aa.lower()]: df_stability.iloc[i]['pr'+aa] for aa in ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU',
                                                                         'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE',
                                                                         'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
        } for i in range(len(df_stability))}

        if wildtype:
            self.wt = wildtype
        else:  # default variant Subtilisin BPN'
            self.wt = 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTN' \
                'AILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'

    def get_stability_score(self, variant_seq):
        aln_refseq = self.wt
        aln_variant_seq = variant_seq

        def cal_log_aa_prob_diff(pos, variant_pos, ref_pos):
            try:
                return math.log(self.pos2aa2prob[pos][aln_variant_seq[variant_pos]]) -\
                       math.log(self.pos2aa2prob[pos][aln_refseq[ref_pos]])
            except:
                print(pos)
                print(aln_variant_seq[variant_pos])
                print(self.pos2aa2prob[pos][aln_variant_seq[variant_pos]])
                print(aln_refseq[ref_pos])
                print(self.pos2aa2prob[pos][aln_refseq[ref_pos]])

        pos, score = 0.0, 0.0
        for i in range(len(aln_refseq)):
            pos += 1
            score += cal_log_aa_prob_diff(pos, i, i)
        return score

    def get_valid_seqs(self, aa_seqs: list, seq_len_flt=221):
        valid_idxes = []
        valid_seqs = []
        for idx, seq in enumerate(aa_seqs):
            # Ignore seq with unknown AAs, and only score for seq with more than 350 AAs
            if len(seq) != seq_len_flt or 'X' in seq or 'B' in seq or ' ' in seq:
                continue
            valid_seqs.append(seq)
            valid_idxes.append(idx)
        return valid_seqs, valid_idxes

    def __call__(self, aa_seqs: list):
        scores = - 10 * np.ones(len(aa_seqs))
        scores_tfd = np.zeros(len(aa_seqs))  # make score for invalid sequence zero by default

        valid_seqs, valid_idxes = self.get_valid_seqs(aa_seqs)
        if len(valid_seqs) > 0:
            scores[np.array(valid_idxes)] = np.array([self.get_stability_score(seq) for seq in valid_seqs])
        return scores

In [20]:
len(test_wt)

221

In [21]:
# prob_scorer = ProtStability()
prob_scorer = ProtStability(prob_path='data/mc/DNase_novozyme.juexiao.csv',)

test['mc_score'] = prob_scorer(test.protein_sequence)

In [22]:
test.head(10)

,seq_id,protein_sequence,pH,data_source,seq_len,mc_score
0,31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,221,1.745531
1,31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,221,1.607645
2,31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes,220,-10.000000
3,31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,221,-0.901497
4,31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,221,-2.253946
5,31395,VPVNPEPDATSVENVALGTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,221,-8.824703
6,31396,VPVNPEPDATSVENVALHTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,221,-0.141061
7,31397,VPVNPEPDATSVENVALITGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,221,-0.098981
8,31398,VPVNPEPDATSVENVALKAGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,221,-1.331715
9,31399,VPVNPEPDATSVENVALKCGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes,221,-2.074183


In [25]:
mc_submission = test[['seq_id', 'mc_score']].rename(columns={"mc_score":"tm"})
mc_submission['tm'] = np.nan_to_num(mc_submission.tm, copy=True, nan=mc_submission.tm.min())
# mc_submission.to_csv('submissions/mc_submission.csv', index=False)
mc_submission.to_csv('submissions/juexiao_submission.csv', index=False)